In [1]:
import yfinance as yf
import numpy as np 
import pandas as pd
import requests
import json
import copy

In [2]:
url = "https://api.usfundamentals.com/v1/companies/xbrl"

In [40]:
contracts = pd.read_csv("data/contracts_data.csv")

In [17]:
contracts["Start Date"]

0        2016-08-01
1        1978-09-30
2        2017-11-17
3        2007-05-09
4        2001-10-26
            ...    
35543    2020-03-31
35544    2019-03-14
35545    2019-01-14
35546    2013-01-06
35547    2011-06-13
Name: Start Date, Length: 35548, dtype: object

In [39]:
contracts_test = contracts[contracts["Start Date"].str.match(r"^\d{4}-\d{2}-\d{2}$", na=False)]
contracts_test["Start Date"] = pd.to_datetime(contracts_test["Start Date"])
contracts_test = contracts_test.sort_values(by="Start Date")

/tmp/ipykernel_45302/1608201159.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contracts_test["Start Date"] = pd.to_datetime(contracts_test["Start Date"])


In [4]:
from groq import Groq

client = Groq(api_key="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")

In [5]:
def isPublic(company): 
    completion = client.chat.completions.create(
        model="gemma2-9b-it",
        messages=[
            {
                "role": "system",
                "content": "Identify publicly traded companies that have a positive correlation with a given company. consider other companies that would benefit from this company thriving, while excluding direct competitors who might not experience the same positive effect. Also, provide any parent or subsidiary companies that are publicly traded, including the company itself if applicable. \n\n# Required Information:\n- Given company name.\n\n# Details to Include:\n- List companies that have a positive correlation with the given company if it receives a large (>100M USD) government contract.\n- Include companies positively influenced by this event or that have a linked dependency, excluding direct competitors as they might not benefit under the same conditions.\n- Include parent or subsidiary companies of the given company, if they are publicly traded.\n- Include the given company itself if it is publicly traded.\n\n# Output Format:\n\nProvide a list ordered by correlation to the given company separated by a comma of their TICKERS ONLY. do not include further information or notes. Return N/A if the company is not publicly traded.\n\n[TICKER]\n\n"
            }, 
            {
                "role": "user",
                "content": f"is {company}"
            },
        ],
        temperature=1.12,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )

    full_response = ""
    for chunk in completion:
        if chunk.choices[0].delta and chunk.choices[0].delta.content:
            full_response+=(chunk.choices[0].delta.content)
    return full_response

In [45]:
filtered_df = contracts_test[
    (contracts_test["Start Date"] >= pd.Timestamp("2016-01-01")) & 
    (contracts_test["Start Date"] <= pd.Timestamp("2023-12-31"))
]
filtered_df

,internal_id,Award ID,Recipient Name,Award Amount,Total Outlays,Description,Contract Award Type,def_codes,COVID-19 Obligations,COVID-19 Outlays,...,Infrastructure Outlays,Awarding Agency,Awarding Sub Agency,Start Date,End Date,recipient_id,prime_award_recipient_id,awarding_agency_id,agency_slug,generated_internal_id
7772,1429690,0025,"LEIDOS, INC.",105433903.26,NaN,IGF::CL::IGF VENDOR MAINTENANCE AGREEMENTS (VMAS),DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Defense,Defense Information Systems Agency,2016-01-01,2016-12-31,32109085-1976-8da6-d5cd-7c8077389b75-C,NaN,1173,department-of-defense,CONT_AWD_0025_9700_HC102812D0021_9700
2233,37476110,W58RGZ16C0008,LOCKHEED MARTIN CORP,356682343.49,43225448.86,"IGF::CL,CT::IGF CONTRACT W58RGZ-16-C-0008 IS H...",DEFINITIVE CONTRACT,['Q'],NaN,NaN,...,NaN,Department of Defense,Department of the Army,2016-01-01,2021-06-30,695d5739-ecc3-8484-ba2b-f9c8b4652a4a-C,NaN,1173,department-of-defense,CONT_AWD_W58RGZ16C0008_9700_-NONE-_-NONE-
15471,775359,0008,NORTHROP GRUMMAN SYSTEMS CORPORATION,54292651.96,NaN,IGF::OT::IGF CONTRACTOR LOGISTICS SUPPORT FOR ...,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Air Force,2016-01-01,2016-12-31,75dec7b1-56eb-4770-da4b-ac736121ab61-C,NaN,1173,department-of-defense,CONT_AWD_0008_9700_FA852313D0002_9700
7205,37394610,W31P4Q16C0028,RAYTHEON COMPANY,114181051.0,NaN,UCA FOR PBL,DEFINITIVE CONTRACT,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Army,2016-01-01,2019-12-31,cce9df51-3950-87f4-0b34-4353d5a08f32-C,NaN,1173,department-of-defense,CONT_AWD_W31P4Q16C0028_9700_-NONE-_-NONE-
10261,32046094,TIRNO11D000240009,"LEIDOS MANAGEMENT SYSTEMS DESIGNERS, INC.",80382264.57,34146605.46,IGF::CT::IGF FOR CRITICAL FUNCTIONS - PAPER AN...,DELIVERY ORDER,"['O', 'Q']",1238280.93,1238280.93,...,NaN,Department of the Treasury,Internal Revenue Service,2016-01-01,2021-06-30,620902f5-ab14-cea5-c2c6-adbe6f4c83f8-C,NaN,456,department-of-the-treasury,CONT_AWD_TIRNO11D000240009_2050_TIRNO11D00024_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33471,167179909,19AQMM24F0158,ATT MOBILITY LLC,26592999.0,NaN,TASK ORDER FOR WIRELESS SERVICES,DELIVERY ORDER,['Q'],NaN,NaN,...,NaN,Department of State,Department of State,2023-12-31,2024-12-30,c0aa4884-6621-8b31-0942-50047a976c83-C,NaN,315,department-of-state,CONT_AWD_19AQMM24F0158_1900_47QTCA19D00MV_4732
12241,155657225,FA862023F4002,L3HARRIS TECHNOLOGIES INTEGRATED SYSTEMS L.P.,68199108.13,NaN,BIG SAFARI,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Air Force,2023-12-31,2023-12-31,14a893b4-32fd-654e-e810-f0ed061e33a2-C,NaN,1173,department-of-defense,CONT_AWD_FA862023F4002_9700_FA862019G4006_9700
13388,170062129,FA807524F0005,"KBR WYLE SERVICES, LLC",62364317.83,NaN,TECHNICAL ANALYSIS FOR THE DEPARTMENT OF THE A...,DELIVERY ORDER,['Q'],NaN,NaN,...,NaN,Department of Defense,Department of the Air Force,2023-12-31,2028-12-30,c1555398-ca16-7024-230c-c902d4f1a351-C,NaN,1173,department-of-defense,CONT_AWD_FA807524F0005_9700_FA807518D0015_9700
9595,166787550,12363N24C4002,"SIERRA TAHOE ENVIRONMENTAL MANAGEMENT, LLC",85643865.96,38615.47,SPIRIT G-Z STEWARDSHIP IRSC - 12363N24C4002 $8...,DEFINITIVE CONTRACT,['Q'],NaN,NaN,...,NaN,Department of Agriculture,Forest Service,2023-12-31,2031-12-31,c9446657-d5f1-02ea-a967-bf137c3b56bf-C,NaN,95,department-of-agriculture,CONT_AWD_12363N24C4002_12C2_-NONE-_-NONE-


In [50]:
import requests

url = "https://data.alpaca.markets/v2/stocks/trades?symbols=AAPL&start=2016-01-03T00%3A00%3A00Z&end=2016-01-04T00%3A00%3A00Z&limit=1000&feed=sip&sort=asc"

headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": "PKB8CFI6U9E0HNO41DSE",
    "APCA-API-SECRET-KEY": "fIe82XWImFfmwureLQEkmEITYibuy2MfHsFMLwqO"
}

response = requests.get(url, headers=headers)

print(response.text)

{"next_page_token":null,"trades":{}}


In [44]:
import finnhub

finnhub_client = finnhub.Client(api_key="ct13df9r01qkcukblrmgct13df9r01qkcukblrn0")
print(finnhub_client.financials_reported(symbol='AAPL', freq='quarterly', startDate="2022-12-31"))

{'cik': '320193', 'data': [{'accessNumber': '0000320193-24-000081', 'symbol': 'AAPL', 'cik': '320193', 'year': 2024, 'quarter': 3, 'form': '10-Q', 'startDate': '2023-10-01 00:00:00', 'endDate': '2024-06-29 00:00:00', 'filedDate': '2024-08-02 00:00:00', 'acceptedDate': '2024-08-01 18:03:34', 'report': {'bs': [{'concept': 'us-gaap_CashAndCashEquivalentsAtCarryingValue', 'unit': 'usd', 'label': 'Cash and cash equivalents', 'value': 25565000000.0}, {'concept': 'us-gaap_MarketableSecuritiesCurrent', 'unit': 'usd', 'label': 'Marketable securities', 'value': 36236000000.0}, {'concept': 'us-gaap_AccountsReceivableNetCurrent', 'unit': 'usd', 'label': 'Accounts receivable, net', 'value': 22795000000}, {'concept': 'us-gaap_NontradeReceivablesCurrent', 'unit': 'usd', 'label': 'Vendor non-trade receivables', 'value': 20377000000}, {'concept': 'us-gaap_InventoryNet', 'unit': 'usd', 'label': 'Inventories', 'value': 6165000000}, {'concept': 'us-gaap_OtherAssetsCurrent', 'unit': 'usd', 'label': 'Other 